In [2]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier

from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import VarianceThreshold
from yellowbrick.features import RFECV

from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


In [3]:
tr_time = pd.read_csv("train_time_series.csv").drop(columns = ["timestamp","UTC time","accuracy"])
tr_lab = pd.read_csv("train_labels.csv").drop(columns = ["timestamp","UTC time"])

In [4]:
tr_time.rename(columns = {"Unnamed: 0":"measurement"}, inplace=True)
tr_lab.rename(columns = {"Unnamed: 0":"measurement"}, inplace=True)

df = tr_time.merge(tr_lab,on="measurement")  #remove / move??

In [5]:
tr_time.tail()

,measurement,x,y,z
3739,24325,0.024384,-0.710709,0.030304
3740,24326,0.487228,-1.099136,-0.015213
3741,24327,0.369446,-0.968506,0.036713
3742,24328,0.167877,-0.802826,0.049805
3743,24329,0.689346,-0.991043,0.034973


In [6]:
tr_lab

,measurement,label
0,20589,1
1,20599,1
2,20609,1
3,20619,1
4,20629,1
...,...,...
370,24289,4
371,24299,4
372,24309,4
373,24319,4


In [7]:
#Standardise X,Y,Z accels
std = preprocessing.StandardScaler()
for k in ["x", "y", "z"]:
    tr_time[k] = std.fit_transform(tr_time[k].values.reshape(-1,1))
tr_time.tail()

,measurement,x,y,z
3739,24325,-0.433313,0.652926,-0.154320
3740,24326,0.736537,-0.208107,-0.284082
3741,24327,0.438839,0.081464,-0.136049
3742,24328,-0.070630,0.448729,-0.098726
3743,24329,1.247394,0.031505,-0.141008


In [25]:
def accel_mag(accels):
    """takes an array containing x,y,z accelerations, calculates their magnitude"""
    return np.sqrt(np.sum(accels**2))

def accel_var(accels):
    """takes an array containing unidirectional accelaration, calculates and returns the variance"""
    return np.var(accels)

def calc_energy(accels, num_observations=10):
    """"""
    return (np.sum(accels**2)/3)
            

In [26]:
#if the above doesn't work aggregate the x's, the y's, the z's togehter - maybe use a sum or mean? then use x, y,z as input X for machine learning
previous = 0

for i in tr_lab.measurement:
    
    
    boolinds = pd.Series((tr_time.measurement.values <= i) & (tr_time.measurement.values > previous))
    num_observations = sum(boolinds)
    step = tr_time[boolinds.values]

    #calculate some values of x, y, z from the step

    sums = []
    means = []
    abs_sums = []
    abs_means = []


    for j in ["x", "y", "z"]:
        accel_sum = np.sum(step[j].values)
        sums.append(accel_sum)
        accel_mean = np.mean(step[j].values)
        means.append(accel_mean)
        abs_sum = np.sum(np.absolute(step[j].values))
        abs_sums.append(abs_sum)
        abs_mean = np.mean(np.absolute(step[j].values))
        abs_means.append(abs_mean)

    tr_lab.loc[tr_lab.measurement == i, "x_sum"] = sums[0]
    tr_lab.loc[tr_lab.measurement == i, "y_sum"] = sums[1]
    tr_lab.loc[tr_lab.measurement == i, "z_sum"] = sums[2]
    tr_lab.loc[tr_lab.measurement == i, "x_mean"] = means[0]
    tr_lab.loc[tr_lab.measurement == i, "y_mean"] = means[1]
    tr_lab.loc[tr_lab.measurement == i, "z_mean"] = means[2]
    tr_lab.loc[tr_lab.measurement == i, "x_abs_sum"] = abs_sums[0]
    tr_lab.loc[tr_lab.measurement == i, "y_abs_sum"] = abs_sums[1]
    tr_lab.loc[tr_lab.measurement == i, "z_abs_sum"] = abs_sums[2]
    tr_lab.loc[tr_lab.measurement == i, "x_abs_mean"] = abs_means[0]
    tr_lab.loc[tr_lab.measurement == i, "y_abs_mean"] = abs_means[1]
    tr_lab.loc[tr_lab.measurement == i, "z_abs_mean"] = abs_means[2]


    #calculate magnitude of acceleration and variation of acceleration for the step - maybe delete later    
    accels = step.loc[:,("x","y","z")]
    mags = accels.apply(accel_mag, axis="columns")
    tr_lab.loc[tr_lab.measurement == i, "sum_accel_mag"] = np.sum(mags)
    tr_lab.loc[tr_lab.measurement == i, "mean_accel_mag"] = np.mean(mags)
    
    tr_lab.loc[tr_lab.measurement == i, "var_x"] = accel_var(accels.x)
    tr_lab.loc[tr_lab.measurement == i, "var_y"] = accel_var(accels.y)
    tr_lab.loc[tr_lab.measurement == i, "var_z"] = accel_var(accels.z)
    tr_lab.loc[tr_lab.measurement == i, "var_mag"] = accel_var(mags)
    
    energy = accels.apply(calc_energy, axis="columns")/num_observations
    tr_lab.loc[tr_lab.measurement == i, "mean_energy"] = np.mean(energy)
    tr_lab.loc[tr_lab.measurement == i, "sum_energy"] = np.sum(energy)
    
    previous = i

In [27]:
classification_target = 'label'
all_covariates = ['x_abs_sum', 'y_abs_sum', 'z_abs_sum', 'sum_accel_mag', 'var_x','var_y', 'var_z', 'var_mag', 'mean_energy']

X_train, X_test, y_train, y_test = train_test_split(tr_lab[all_covariates], tr_lab[classification_target])


In [28]:
knn = KNeighborsClassifier()

parameters = {
    "n_neighbors":np.arange(0,100,5),
    "weights":['uniform', 'distance']
}

cv = GridSearchCV(knn,parameters,cv=5)
cv.fit(X_train,y_train)
best_params = cv.best_params_ 
best_params

{'n_neighbors': 10, 'weights': 'uniform'}

In [29]:
#initialise the random forest model with best hyperparameters and train
knn = KNeighborsClassifier(n_neighbors = best_params['n_neighbors'], weights = best_params['weights'])
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=10)

In [30]:
print("Accuracy on training set: {:.3f}".format(knn.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(knn.score(X_test, y_test)))

Accuracy on training set: 0.694
Accuracy on test set: 0.691


SyntaxError: invalid syntax (<ipython-input-17-c1cef89704da>, line 1)